In [ ]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.augment_utils.add_noise import AddNoise
from IPython.display import Audio, display
from pathlib import Path
import torch
import torchaudio
import numpy as np

In [ ]:
folder = Path('/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/rir/Audio')

wavs = []
srs = []

for path in folder.iterdir():
    if path.suffix != '.wav':
        continue
    wav, sr = torchaudio.load(path)
    wav = wav.squeeze()
    wav = wav / wav.norm()
    wavs.append(wav)
    srs.append(sr)
sr = srs[0]


In [ ]:
from matplotlib import pyplot as plt

def plot(wav):
    plt.plot(np.linspace(0, len(wav) / sr, wav.shape[-1]), wav)

plot(wavs[3])

In [ ]:
dataset = L2ArcticDataset(
    "train",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/",
)

In [ ]:
def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    display(Audio(waveform, rate=sample_rate))


audio = [
    torchaudio.functional.resample(dataset[0][0], orig_freq=44100, new_freq=16000),
    torchaudio.functional.resample(dataset[1][0], orig_freq=44100, new_freq=16000),
]

In [ ]:
Audio(torchaudio.functional.fftconvolve(audio[0], wavs[30]), rate=16000)

In [ ]:
Audio(torchaudio.functional.fftconvolve(audio[0], wav[0]), rate=16000)

In [ ]:
Audio(audio[0], rate=16000)

In [ ]:
def plot_rev(x):
    plot(torchaudio.functional.fftconvolve(audio[0], wavs[x]))

In [ ]:
rng = torch.Generator()
rng.manual_seed(1337)
n = 5
indices = torch.randint(
    0,
    len(wavs),
    (n,),
    generator=rng,
)

fig, ax = plt.subplots(figsize=(10, 2 * n))

for i, idx in enumerate(indices):
    plt.plot(torchaudio.functional.fftconvolve(audio[0], wavs[idx]))
    plt.axis('off')
    plt.savefig(f"wav_{i}.png", bbox_inches='tight')
    plt.clf()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 2))
plt.plot(audio[0], color='k')
plt.axis('off')
plt.savefig(f"wav.png", bbox_inches='tight')
plt.clf()

In [ ]:
plot(audio[0])

In [ ]:
play_audio(audio[0], 16000)

In [ ]:
augmentation = AddNoise(
    prob=1.0,
    snr_low=0,
    snr_high=0.4,
    device="cpu",
    seed=1337,
)
audio = torch.nn.utils.rnn.pad_sequence(audio, batch_first=True)
lens = torch.tensor([len(audio[0]), len(audio[1])], dtype=torch.long)
aug_audio =     augmentation(
        audio,
        lens,
    )
play_audio(
    aug_audio[0],
    sample_rate=16000,
)

audio - aug_audio